<a href="https://colab.research.google.com/github/rdkdaniel/NLP-Projects/blob/main/Automatic_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://neurondai.medium.com/automatic-text-summarization-system-using-transformers-are-you-tired-of-reading-a-long-paper-22d2cd9f5260

In [1]:
#Libararies/dependencies
!pip install transformers
!pip install newspaper3k
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.3 MB/s 
     |████████████████████████████████| 163 kB 51.2 MB/s 
     |████████████████████████████████| 6.6 MB 27.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 7.9 MB/s 
     |████████████████████████████████| 7.4 MB 46.5 MB/s 
     |████████████████████████████████| 93 kB 1.0 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13553 sha256=d936758623bc88c4e958100b87af6d17e0a6269d08f4715b2b22f71275329461
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3357 sha256=67443a6cccbb1dc483efe1b1837cff1bd56e574

In [2]:
import os
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [3]:
import torch
import streamlit as st
import time

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
2022-10-07 06:37:02.773 INFO    numexpr.utils: NumExpr defaulting to 2 threads.


In [6]:
print(torch.cuda.is_available())

False


**torch maybe the issue**

In [10]:
from newspaper import Articleif torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cuda")st.title('Text Summarization Demo')
st.markdown('Orient Development team')
model = st.selectbox('Model', ["Bart","Pegasus"])link_paper = st.text_area('https://e.vnexpress.net/news/news/pm-orders-covid-19-inoculation-starting-this-week-4242380.html')
article = Article(link_paper)
article.download()
article.parse()
input_text = article.textst.text(input_text)def run_model(input_text):
    start_time = time.time()
    if model == "Bart":
        bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)
        bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
    
        input_text = ' '.join(input_text.split())
        input_tokenized = bart_tokenizer.encode(input_text, return_tensors='pt').to(device)summary_ids = bart_model.generate(input_tokenized,
                                    num_beams = 4,
                                    num_return_sequences = 1,
                                    no_repeat_ngram_size = 2,
                                    length_penalty = 1,
                                    min_length = 12,
                                    max_length = 128,
                                    early_stopping = True)
    
        output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
        st.write('Summary')
        st.success(output)
    else:
        pegasus_model = PegasusForConditionalGeneration.from_pretrained("google/Pegasus-cnn_dailymail").to(device)
        pegasus_tokenizer = PegasusTokenizer.from_pretrained("google/Pegasus-cnn_dailymail")
        
        input_text = ' '.join(input_text.split())
        batch = pegasus_tokenizer.prepare_seq2seq_batch(input_text, truncation=True, padding='longest', return_tensors="pt").to(device)
        
        summary_ids = pegasus_model.generate(**batch,
                                            num_beams=6,
                                            num_return_sequences=1,
                                            no_repeat_ngram_size = 2,
                                            length_penalty = 1,
                                            min_length = 30,
                                            max_length = 128,
                                            early_stopping = True)
        
        output = [pegasus_tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)]
        st.write("Summary")
        st.success(output)
    print("--- %s seconds ---" % (time.time() - start_time))
    
if st.button('Submit'):
    run_model(input_text)

SyntaxError: ignored

**Check on this one**